In [1]:
import imp
import pandas as pd

# Generate overall summary table - 2016

Table 3 of the 2015 report summarises all monitored and modelled loads for the whole of Norway, and for the four OSPAR regions. This notebook reads the output from the processing of the observed data (see [here](http://nbviewer.jupyter.org/github/JamesSample/rid/blob/master/notebooks/rid_working_2016-17.ipynb)) and the modelling using NOPE (see [here](http://nbviewer.jupyter.org/github/JamesSample/rid/blob/master/notebooks/loads_unmonitored_regions_2016.ipynb)) and generates a Word version of Table 3. The code is based on the exploratory work [here](http://nbviewer.jupyter.org/github/JamesSample/rid/blob/master/notebooks/word_data_tables.ipynb).

In [2]:
# Import custom RID functions
rid_func_path = (r'C:\Data\James_Work\Staff\Oeyvind_K\Elveovervakingsprogrammet'
                 r'\Python\rid\notebooks\useful_rid_code.py')

rid = imp.load_source('useful_rid_code', rid_func_path)

# Import model
nope_path = (r'C:\Data\James_Work\Staff\Oeyvind_K\Elveovervakingsprogrammet'
             r'\Python\rid\notebooks\nope.py')
nope = imp.load_source('nope', nope_path)

## 1. Summarise monitoring data

In [3]:
# Read station data
in_xlsx = (r'C:\Data\James_Work\Staff\Oeyvind_K\Elveovervakingsprogrammet'
           r'\Data\RID_Sites_List.xlsx')
stn_df = pd.read_excel(in_xlsx, sheetname='RID_All')

# Get just cols of interest and drop duplicates 
# (some sites are in the same regine)
stn_df = stn_df.drop_duplicates(subset=['ospar_region', 'nve_vassdrag_nr'])

# Get catch IDs with calib data
calib_nds = set(stn_df['nve_vassdrag_nr'].values)

# Build network
in_path = (r'C:\Data\James_Work\Staff\Oeyvind_K\Elveovervakingsprogrammet'
           r'\NOPE\NOPE_Annual_Inputs\nope_input_data_1990.csv')
g, nd_list = nope.build_calib_network(in_path, calib_nds)

# Get list of downstream nodes
ds_nds = []
for nd in g:
    # If no downstream nodes
    if g.out_degree(nd) == 0:
        # Node is of interest
        ds_nds.append(nd)

# Get just the downstream catchments
stn_df = stn_df[stn_df['nve_vassdrag_nr'].isin(ds_nds)]

**Note:** Remember to update the year below.

In [4]:
# Read data
in_csv = (r'C:\Data\James_Work\Staff\Oeyvind_K\Elveovervakingsprogrammet'
          r'\Results\Loads_CSVs\loads_and_flows_all_sites_2016.csv')
mon_df = pd.read_csv(in_csv)

# Get just the downstream catchments
#mon_df = mon_df[mon_df['station_id'].isin(stn_df['station_id'].values)]

# Group by OSPAR region
mon_df1 = mon_df.groupby(['ospar_region', 'rid_group']).sum()

# Totals for Norway
mon_df2 = mon_df.groupby('rid_group').sum().reset_index()
mon_df2['ospar_region'] = 'NORWAY'
mon_df2.set_index(['ospar_region', 'rid_group'], inplace=True)

# Combine
mon_df = pd.concat([mon_df1, mon_df2], axis=0)

# Cols of interest
cols = [i for i in mon_df.columns if i.split('_')[1] != 'Est']
mon_df = mon_df[cols]
del mon_df['station_id']

# Rename cols to match template
mon_df['Flow rate_1000m3/day'] = mon_df['mean_q_1000m3/day']
del mon_df['mean_q_1000m3/day']

# Units are correct, so remove
mon_df.columns = [i.split('_')[0] for i in mon_df.columns]

mon_df.round(0)

C:\Data\WinPython-64bit-2.7.10.3\python-2.7.10.amd64\lib\site-packages\pandas\core\series.py:1342: RuntimeWarning: invalid value encountered in rint
  result = _values_from_object(self).round(decimals)


Ag   As   Cd    Cr    Cu    Hg  NH4-N  \
ospar_region        rid_group                                           
LOFOTEN-BARENTS SEA rid_108    NaN  1.0  0.0   1.0   6.0   5.0   70.0   
                    rid_11     0.0  1.0  0.0   1.0   2.0   1.0   38.0   
                    rid_36     0.0  2.0  0.0   4.0  33.0   2.0  134.0   
NORTH SEA           rid_108    NaN  1.0  0.0   1.0  10.0  11.0  168.0   
                    rid_11     0.0  0.0  0.0   0.0   1.0   2.0   26.0   
                    rid_36     0.0  2.0  0.0   3.0  11.0  20.0  156.0   
NORWEGIAN SEA2      rid_108    NaN  1.0  0.0   1.0   5.0   8.0  115.0   
                    rid_11     0.0  1.0  0.0   2.0   9.0   3.0    8.0   
                    rid_36     0.0  2.0  0.0   7.0  18.0  16.0   82.0   
SKAGERAK            rid_108    NaN  1.0  0.0   1.0   2.0   3.0   73.0   
                    rid_11     0.0  8.0  1.0  10.0  50.0  38.0  594.0   
                    rid_36     0.0  2.0  0.0   1.0   5.0   7.0   99.0   
NORWAY              rid_108    NaN  4.0  0.0   4.0  24.0  27.0  426.0   
                    rid_11     0.0  9.0  1.0  14.0  63.0  43.0  665.0   
                    rid_36     0.0  8.0  1.0  15.0  67.0  45.0  470.0   

                                 NO3-N     Ni  PO4-P    Pb       SPM  \
ospar_region        rid_group                                          
LOFOTEN-BARENTS SEA rid_108      178.0    5.0   11.0   0.0    8879.0   
                    rid_11       148.0    1.0   11.0   0.0    6927.0   
                    rid_36       528.0  155.0   46.0   1.0   74207.0   
NORTH SEA           rid_108     3086.0    7.0   25.0   3.0   25238.0   
                    rid_11       351.0    1.0    6.0   0.0    3529.0   
                    rid_36      3717.0    5.0   81.0   5.0   67869.0   
NORWEGIAN SEA2      rid_108      780.0    4.0   18.0   1.0   21594.0   
                    rid_11       287.0    4.0   24.0   1.0   36002.0   
                    rid_36      1831.0   16.0   80.0   2.0   88349.0   
SKAGERAK            rid_108     1321.0    2.0   14.0   1.0   10791.0   
                    rid_11     11883.0   28.0  321.0  21.0  385715.0   
                    rid_36      1387.0    3.0   13.0   3.0   12374.0   
NORWAY              rid_108     5365.0   17.0   68.0   5.0   66502.0   
                    rid_11     12669.0   34.0  362.0  22.0  432174.0   
                    rid_36      7464.0  179.0  221.0  11.0  242799.0   

                                   SiO2       TOC     TOTN   TOTP     Zn  \
ospar_region        rid_group                                              
LOFOTEN-BARENTS SEA rid_108     21610.0   23194.0   1006.0   19.0    6.0   
                    rid_11      16126.0   15159.0    862.0   25.0    1.0   
                    rid_36      80641.0   67688.0   5871.0  152.0   16.0   
NORTH SEA           rid_108     27778.0   27976.0   5021.0   91.0   29.0   
                    rid_11       3056.0    4317.0    796.0   21.0    6.0   
                    rid_36      32823.0   44560.0   6893.0  167.0   86.0   
NORWEGIAN SEA2      rid_108     18374.0   20205.0   1824.0   59.0    8.0   
                    rid_11      10365.0   13856.0   1053.0   51.0   18.0   
                    rid_36      46938.0   63117.0   5093.0  141.0   35.0   
SKAGERAK            rid_108      8392.0   13975.0   2033.0   61.0   17.0   
                    rid_11     150700.0  169674.0  21421.0  571.0  282.0   
                    rid_36      19898.0   40915.0   3335.0   48.0   47.0   
NORWAY              rid_108     76152.0   85350.0   9883.0  230.0   61.0   
                    rid_11     180246.0  203006.0  24132.0  667.0  307.0   
                    rid_36     180299.0  216280.0  21192.0  508.0  184.0   

                               Flow rate  
ospar_region        rid_group             
LOFOTEN-BARENTS SEA rid_108      28033.0  
                    rid_11       10268.0  
                    rid_36       47266.0  
NORTH SEA           rid_108      69709.0  
              

## 2. Data for unmonitored areas

**Note:** Remember to update the year below.

In [5]:
# Read data
in_csv = (r'C:\Data\James_Work\Staff\Oeyvind_K\Elveovervakingsprogrammet'
          r'\Results\Unmon_loads\unmon_loads_2016.csv')
umon_df = pd.read_csv(in_csv, index_col=0)

# Rename cols
umon_df.columns = [i.replace('RENSEANLEGG', 'sew') for i in umon_df.columns]
umon_df.columns = [i.replace('INDUSTRI', 'ind') for i in umon_df.columns]
umon_df.columns = [i.replace('_tonn', '') for i in umon_df.columns]
umon_df.columns = [i.replace('AQUAKULTUR', 'fish') for i in umon_df.columns]

# Convert Hg to kgs
umon_df['sew_Hg'] = umon_df['sew_Hg']*1000
umon_df['ind_Hg'] = umon_df['ind_Hg']*1000

umon_df.round(0)

,flow,sew_n,sew_p,ind_n,ind_p,fish_n,fish_p,diff_n,diff_p,sew_po4,...,sew_S.P.M.,sew_As,sew_Pb,sew_Cd,sew_Cu,sew_Zn,sew_Ni,sew_Cr,sew_Hg,fish_Cu
ospar_region,,,,,,,,,,,,,,,,,,,,,
NORWAY,408822.0,13615.0,1128.0,2354.0,246.0,56691.0,9781.0,34781.0,711.0,677.0,...,1927.0,0.0,0.0,0.0,4.0,12.0,2.0,1.0,5.0,1088.0
LOFOTEN-BARENTS SEA,104554.0,1226.0,177.0,83.0,6.0,14141.0,2435.0,4980.0,102.0,106.0,...,120.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,271.0
NORTH SEA,151022.0,3714.0,440.0,412.0,90.0,19561.0,3370.0,14505.0,239.0,264.0,...,912.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,1.0,375.0
NORWEGIAN SEA2,143758.0,2828.0,392.0,984.0,109.0,22964.0,3971.0,12382.0,281.0,235.0,...,891.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,442.0
SKAGERAK,9488.0,5847.0,120.0,875.0,41.0,25.0,4.0,2915.0,88.0,72.0,...,4.0,0.0,0.0,0.0,2.0,8.0,1.0,0.0,4.0,0.0


## 3. Create table

In [6]:
# Create table 3
in_docx = (r'C:\Data\James_Work\Staff\Oeyvind_K\Elveovervakingsprogrammet'
           r'\Results\Word_Tables\2017Analysis_2016Data\Table_3_2016.docx')

rid.write_word_overall_table(mon_df, umon_df, in_docx)

Finished.
